In [54]:
#Importing Dependencies
!pip install tensorflow 
!pip install tensorflow-gpu 
!pip install opencv-python
!pip install mediapipe
!pip install --upgrade pip
!pip install sklearn 
!pip install matplotlib 



  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached python_version-0.0.2-py2.py3-none-any.whl (3.4 kB)
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [18 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\91842\AppData\Local\Temp\pip-install-0bksg6yf\tensorflow-gpu_4b2f5b3d9b1e4eb486103645894717da\setup.py", line 37, in <module>
      raise Exception(TF_REMOVAL_WARNING)
  Exception:
  
  The "tensorflow-gpu" package has been removed!
  
  Please install "tensorflow" instead.
  
  Other than the name, the two packages have been identical
  since TensorFlow 2.1, or roughly since Sep 2019. For more
  information, see: pypi.org/project/tensorflow-gpu
  
  
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
ERROR: Could not build wheels for tensorflow-gpu, which is required to install pyproject.toml-

In [55]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [56]:
import cv2 
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from cv2 import VideoCapture

In [57]:
mp_holistic = mp.solutions.holistic  #Holistic
mp_drawing = mp.solutions.drawing_utils # Drawing Utitlities

In [58]:
def mediapipe_detection(image, model) :
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #Color conversion
    image.flags.writeable = False                  #Image is no longer writable 
    results = model.process(image)                 #make prediction
    image.flags.writeable = True                   #Image is writable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #Color conversion back
    return image, results

In [59]:
def draw_landmarks(image, results) : 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [60]:
def draw_styled_landmarks(image, results) : 
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

In [61]:
#Access the webcam
cap = VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic : 
    while cap.isOpened() :

        #Reading frames
        ret, frame = cap.read()

        #Make detection 
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        #print(results)

        #show to screen
        cv2.imshow('OpenCV Feed', image)

        #break 
        if cv2.waitKey(10) & 0xFF == ord('q') : 
            break

    cap.release()
    cv2.destroyAllWindows()

In [9]:
np.array(results.pose_landmarks.landmark).shape

(33,)

In [10]:
np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().shape

(132,)

In [40]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132*4)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [41]:
def extract_keypoints(results) : 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [13]:
extract_keypoints(results).shape

(1662,)

In [14]:
results.face_landmarks

landmark {
  x: 0.5892338752746582
  y: 0.6734254956245422
  z: -0.01998540572822094
}
landmark {
  x: 0.5881712436676025
  y: 0.6409987211227417
  z: -0.04003647714853287
}
landmark {
  x: 0.5885950922966003
  y: 0.6504939198493958
  z: -0.020362937822937965
}
landmark {
  x: 0.580742597579956
  y: 0.6024344563484192
  z: -0.03075527958571911
}
landmark {
  x: 0.5878292918205261
  y: 0.629459798336029
  z: -0.042912133038043976
}
landmark {
  x: 0.5874143838882446
  y: 0.6136247515678406
  z: -0.0402703732252121
}
landmark {
  x: 0.5865552425384521
  y: 0.5746620297431946
  z: -0.020563803613185883
}
landmark {
  x: 0.5316612720489502
  y: 0.5732002258300781
  z: 0.009231550619006157
}
landmark {
  x: 0.5856572985649109
  y: 0.5435433983802795
  z: -0.015830490738153458
}
landmark {
  x: 0.5852187275886536
  y: 0.5276533365249634
  z: -0.01786905899643898
}
landmark {
  x: 0.58360356092453
  y: 0.4743117094039917
  z: -0.011898618191480637
}
landmark {
  x: 0.5893594026565552
  y: 0.6

In [21]:
#path for exported array
DATA_PATH = os.path.join('MP_Data')

#Actions that we are going to detect
actions = np.array(['hello', 'thanks', 'You`r Welcome'])

# 30 videos worth of data
no_sequences = 30 

#30 frames of length
sequence_length = 30 

In [22]:
for action in actions : 
    for sequence in range(no_sequences) : 
        try : 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except : 
            pass

In [31]:
cap = VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic : 
    #Loop through actions 
    for action in actions :
        #Loop through the collection of frames
        for sequence in range(no_sequences) : 
            
            #Loop through each frame
            for frame_num in range(sequence_length) : 
                
                #Reading frames
                ret, frame = cap.read()

                #Make detection 
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                #print(results)
                
                
                #collecting images 
                
                if frame_num == 0 : 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} video number {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 4, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else : 
                    cv2.putText(image, f'Collecting frames for {action} video number {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                
                #show to screen
                cv2.imshow('OpenCV Feed', image)
                
                #Saving the arrays
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                #break 
                if cv2.waitKey(10) & 0xFF == ord('q') : 
                    break

    cap.release()
    cv2.destroyAllWindows()

In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [24]:
label_map = {label : num for num, label in enumerate(actions)}

In [25]:
label_map

{'hello': 0, 'thanks': 1, 'You`r Welcome': 2}

In [26]:
#combining all sequences 

sequences = []
labels = []
for action in actions : 
    for sequence in range(no_sequences) : 
        window = []
        for frame_num in range(sequence_length) : 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f'{str(frame_num)}.npy'))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [27]:
np.array(sequences).shape

(90, 30, 1662)

In [28]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [29]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7)

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [36]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [37]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation= 'relu', input_shape = (30,1662)))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(64, return_sequences=False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [38]:
model.compile(optimizer= 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [39]:
model.fit(X_train, y_train , epochs = 245, callbacks=[tb_callback])

Epoch 1/245
1/1 [==============================] - 6s 6s/step - loss: 1.1233 - categorical_accuracy: 0.2963
Epoch 2/245
1/1 [==============================] - 0s 83ms/step - loss: 1.9692 - categorical_accuracy: 0.2963
Epoch 3/245
1/1 [==============================] - 0s 137ms/step - loss: 1.1047 - categorical_accuracy: 0.6667
Epoch 4/245
1/1 [==============================] - 0s 120ms/step - loss: 1.1104 - categorical_accuracy: 0.2222
Epoch 5/245
1/1 [==============================] - 0s 121ms/step - loss: 1.0538 - categorical_accuracy: 0.3333
Epoch 6/245
1/1 [==============================] - 0s 116ms/step - loss: 1.1794 - categorical_accuracy: 0.4074
Epoch 7/245
1/1 [==============================] - 0s 111ms/step - loss: 1.5861 - categorical_accuracy: 0.6667
Epoch 8/245
1/1 [==============================] - 0s 130ms/step - loss: 1.1959 - categorical_accuracy: 0.0741
Epoch 9/245
1/1 [==============================] - 0s 130ms/step - loss: 1.9727 - categorical_accuracy: 0.2963
Epoch

1/1 [==============================] - 0s 123ms/step - loss: 1.0661e-04 - categorical_accuracy: 1.0000
Epoch 146/245
1/1 [==============================] - 0s 105ms/step - loss: 1.0457e-04 - categorical_accuracy: 1.0000
Epoch 147/245
1/1 [==============================] - 0s 112ms/step - loss: 1.0260e-04 - categorical_accuracy: 1.0000
Epoch 148/245
1/1 [==============================] - 0s 92ms/step - loss: 1.0070e-04 - categorical_accuracy: 1.0000
Epoch 149/245
1/1 [==============================] - 0s 128ms/step - loss: 9.8850e-05 - categorical_accuracy: 1.0000
Epoch 150/245
1/1 [==============================] - 0s 143ms/step - loss: 9.7067e-05 - categorical_accuracy: 1.0000
Epoch 151/245
1/1 [==============================] - 0s 113ms/step - loss: 9.5346e-05 - categorical_accuracy: 1.0000
Epoch 152/245
1/1 [==============================] - 0s 111ms/step - loss: 9.3677e-05 - categorical_accuracy: 1.0000
Epoch 153/245
1/1 [==============================] - 0s 129ms/step - loss: 9.20

In [42]:
# evaluate the model accuarcy
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

2/2 [==============================] - 1s 45ms/step - loss: 0.3719 - categorical_accuracy: 0.9683
Accuracy: 0.9682539701461792


In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [44]:
res= model.predict(X_test)

2/2 [==============================] - 1s 59ms/step


In [45]:
res

array([[9.99937057e-01, 4.02960941e-05, 2.25927888e-05],
       [1.40863619e-06, 9.99996543e-01, 2.01568560e-06],
       [9.99911666e-01, 3.90874811e-05, 4.92714971e-05],
       [6.15171757e-07, 9.99998569e-01, 8.25615359e-07],
       [2.97504528e-08, 2.87542018e-10, 1.00000000e+00],
       [2.52209566e-06, 9.99992847e-01, 4.60082174e-06],
       [8.51024306e-05, 1.98212305e-07, 9.99914646e-01],
       [9.99939442e-01, 3.89515881e-05, 2.15706423e-05],
       [1.59525122e-08, 1.61983663e-10, 1.00000000e+00],
       [1.00692550e-05, 9.99957561e-01, 3.23911590e-05],
       [2.51836667e-07, 9.99999523e-01, 2.73640751e-07],
       [3.26491204e-07, 9.99999285e-01, 3.54218372e-07],
       [6.03737192e-07, 9.99998689e-01, 6.99928080e-07],
       [1.39912788e-06, 9.99995708e-01, 2.86926115e-06],
       [9.99937892e-01, 3.52973584e-05, 2.68233034e-05],
       [9.99923706e-01, 5.31384721e-05, 2.31017129e-05],
       [2.11926590e-07, 9.99999523e-01, 1.85005106e-07],
       [9.99936223e-01, 4.28929

In [46]:
actions[np.argmax(res[2])]

'hello'

In [47]:
actions[np.argmax(y_test[2])]

'hello'

In [48]:
model.save('action.h5')

In [49]:
# del model

In [50]:
model.load_weights('action.h5')

In [51]:
# res = []
sequence = []
sentence = []
threshold = 0.4

In [52]:
actions[np.argmax(res)]

IndexError: index 14 is out of bounds for axis 0 with size 3

In [53]:
cap = VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic : 
    while cap.isOpened() :

        #Reading frames
        ret, frame = cap.read()

        #Make detection 
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        #print(results)
        
        #determining the result
        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30 :
            print(np.array(sequence).shape)
            res= model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            #SHOWING THE RESULTS

            #checking if the value of the predicted action is greater than the threshold
            if res[np.argmax(res)] > threshold: 

                #checking if the sentence has more than word
                if len(sentence) > 0: 

                    #if the predicted word doesn't match the last word, add it
                    if actions[np.argmax(res)] != sentence[-1] : 
                        sentence.append(actions[np.argmax(res)])

                    #if the predicted word matches the last word 
                else : 
                    sentence.append(actions[np.argmax(res)])

        #if the length of the sentence is greater than 5
        if len(sentence) > 5 : 

            #reduce the array to the last five values
            sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640,40), (245,117,16) , -1)
        cv2.putText(image, ' '.join(sentence) , (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1 , (255,255,255), 1,  cv2.LINE_AA)

        #show to screen
        cv2.imshow('OpenCV Feed', image)


        #break 
        if cv2.waitKey(10) & 0xFF == ord('q') : 
            break

    cap.release()
    cv2.destroyAllWindows()

(30, 1662)
1/1 [==============================] - 0s 23ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 10ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 11ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 14ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 11ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 17ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 14ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 9ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 19ms/step
thanks
(30, 1662)
1/1 [=====

1/1 [==============================] - 0s 16ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 16ms/step
thanks


In [92]:
cap.release()
cv2.destroyAllWindows()